In [1]:
import numpy as np
import random
import time
from IPython.display import clear_output
from copy import copy, deepcopy

In [2]:
EMPTY_CELL = 0
WALL = 1
PLAYER_ON_EMPTY_CELL = 2
PORTAL = 3
PLAYER_ON_PORTAL = 5
ENEMY_ANT = 6

PLAYER_CONTAINER_CELLS = [PLAYER_ON_EMPTY_CELL, PLAYER_ON_PORTAL]
PLAYER_MOVABLE_CELLS = [EMPTY_CELL, PORTAL]
ENEMY_MOVABLE_CELLS = [EMPTY_CELL, PLAYER_ON_EMPTY_CELL]

In [3]:
def encode_map(m):
    ret = ''
    for r in m:
        ret = ret + ''.join([str(c) for c in r])
    return ret

In [4]:
def decode_map(s, size):
    ret = []
    for j in range(size):
        row = []
        for i in range(size):
            row.append(s[j*size + i])
        ret.append(row)
    return ret

In [5]:
def print_map(m):
    for row in m:
        print(row)

In [6]:
# expose same api with gym 
# so I dont need to change the training code

class Space:
    def __init__(self, n):
        self.n = n
    def sample(self):
        return random.randint(0, self.n-1)

class WatchYourBack:
    def __init__(self, original_map, states):
        self.states = states
        self.original_map = original_map
        self.map = None
        self.action_space = Space(4)
        self.observation_space = Space(len(states))
        self.reset()
    
    def reset(self):
        self.map = deepcopy(self.original_map)
        return self.states[encode_map(self.map)]
        
    def get_player_pos(self):
        for j in range(len(self.map)):
            for i in range(len(self.map[0])):
                if self.map[j][i] in PLAYER_CONTAINER_CELLS:
                    return [j, i]

    def render(self):
        print_map(self.map)
                
    def close(self):
        return 0
    
    def move_enemies(self):
        [player_pos_y, player_pos_x] = self.get_player_pos()
    
    def step(self, action):
        # new_state, reward, done, info = env.step(action)
        #
        # 0: up, 1: right, 2: down, 3: left
        
        done = False
        new_state = None
        reward = 0
        info = None
        
        xy = {
            0: [0, -1],
            1: [1,  0],
            2: [0,  1],
            3: [-1, 0]
        }
        
        [delta_x, delta_y] = xy[action]
        [player_pos_y, player_pos_x] = self.get_player_pos()
        [target_pos_y, target_pos_x] = [player_pos_y + delta_y, player_pos_x + delta_x]
        
        if target_pos_x >= 0 and target_pos_x < len(self.map[0]) and \
            target_pos_y >= 0 and target_pos_y < len(self.map) and \
            self.map[target_pos_y][target_pos_x] in PLAYER_MOVABLE_CELLS:

            targetCellId = self.map[target_pos_y][target_pos_x]
            self.map[player_pos_y][player_pos_x] = 0

            if targetCellId == 3:
                # hit the goal
                reward = 1
                done = True
                self.map[target_pos_y][target_pos_x] = 5
                info = "hit the goal!"
            else:
                self.map[target_pos_y][target_pos_x] = 2
                info = f"player moved to {target_pos_y} {target_pos_x}"
        
        else:
            info = "player could not move"
            
        # each state has a number value
        new_state = self.states[encode_map(self.map)]
        
        return new_state, reward, done, info

In [22]:
def create_states_for_playerless_map(m, enemy_count=0):
    
    # player movements
    states = []
    for j in range(len(m)):
        for i in range(len(m[0])):
            if m[j][i] == 0:
                copy_m = deepcopy(m)
                copy_m[j][i] = 2 # move player to there
                states.append(encode_map(copy_m))
        
    # enemies 
    states_with_enemy = []    
    for s in states:
        for cellIndex in range(len(s)):
            cell = int(s[cellIndex])
            if cell == EMPTY_CELL:
                copy_s = s[:cellIndex] + '6' + s[cellIndex+1:]
                states_with_enemy.append(copy_s)

    # player reaching the goal
    states_with_goal = states_with_enemy[:]
    for s in states_with_enemy:
        for cellIndex in range(len(s)):
            cell = int(s[cellIndex])
            if cell == 3:
                copy_s = s[:cellIndex] + '5' + s[cellIndex+1:]
                copy_s = copy_s.replace('2', '0')
                states_with_goal.append(copy_s)

    states_with_goal_unq = list(set(states_with_goal))
    dct = {}
    for index in range(len(states_with_goal_unq)):
        dct[states_with_goal_unq[index]] = index
    return dct

In [23]:
states = create_states_for_playerless_map([
    [1, 0, 3],
    [0, 0, 0],
    [0, 0, 0]
], enemy_count = 1)

print(states)

original_map = [
    [1, 0, 3],
    [6, 0, 0],
    [0, 0, 2]
]

env = WatchYourBack(original_map, states)

{'163000020': 0, '103060200': 1, '123006000': 2, '103002060': 3, '123000060': 4, '103000026': 5, '103000602': 6, '103206000': 7, '103060002': 8, '103020060': 9, '163000002': 10, '103600200': 11, '103260000': 12, '103020006': 13, '103600020': 14, '105000060': 15, '163002000': 16, '103026000': 17, '103006020': 18, '103002600': 19, '103000620': 20, '163200000': 21, '103000206': 22, '123000600': 23, '103020600': 24, '163020000': 25, '103600002': 26, '123060000': 27, '103062000': 28, '103060020': 29, '103602000': 30, '103200060': 31, '103620000': 32, '103006200': 33, '103000062': 34, '105600000': 35, '103000260': 36, '103200600': 37, '103006002': 38, '123600000': 39, '105006000': 40, '165000000': 41, '105060000': 42, '105000006': 43, '103200006': 44, '105000600': 45, '103002006': 46, '123000006': 47, '163000200': 48}


In [24]:
# Initialize q-table
# 
# rows: state space in the environment
# columns: action space.

In [25]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [26]:
# Setup hyperparameters
# 
# 

num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [27]:
rewards_all_episodes = []

# Q-learning algorithm
for episode in range(num_episodes):
    # initialize new episode params
    state = env.reset()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        
        # explore or exploit?
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])  # exploit: get highest q-value move
        else:
            action = env.action_space.sample()    # explore: select a random move
        
        # Take new action
        new_state, reward, done, info = env.step(action)

        print(new_state, reward, done, info)
        
        # Update Q-table
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        # Set new state
        state = new_state

        # Add new reward        
        rewards_current_episode += reward 

        if done == True: 
            break

    # Exploration rate decay   
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)


14 0 False player moved to 2 1
14 0 False player could not move
14 0 False player could not move
32 0 False player moved to 1 1
30 0 False player moved to 1 2
35 1 True hit the goal!
26 0 False player could not move
30 0 False player moved to 1 2
32 0 False player moved to 1 1
30 0 False player moved to 1 2
26 0 False player moved to 2 2
30 0 False player moved to 1 2
32 0 False player moved to 1 1
30 0 False player moved to 1 2
26 0 False player moved to 2 2
30 0 False player moved to 1 2
30 0 False player could not move
26 0 False player moved to 2 2
30 0 False player moved to 1 2
30 0 False player could not move
26 0 False player moved to 2 2
14 0 False player moved to 2 1
26 0 False player moved to 2 2
14 0 False player moved to 2 1
32 0 False player moved to 1 1
32 0 False player could not move
14 0 False player moved to 2 1
11 0 False player moved to 2 0
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not 

11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
14 0 False player moved to 2 1
26 0 False player moved to 2 2
14 0 False player moved to 2 1
26 0 False player moved to 2 2
26 0 False player could not move
26 0 False player could not move
26 0 False player could not move
14 0 False player moved to 2 1
11 0 False player moved to 2 0
14 0 False player moved to 2 1
11 0 False player moved to 2 0
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could not move
11 0 False player could no

30 0 False player moved to 1 2
32 0 False player moved to 1 1
32 0 False player could not move
14 0 False player moved to 2 1
26 0 False player moved to 2 2
14 0 False player moved to 2 1
11 0 False player moved to 2 0
14 0 False player moved to 2 1
26 0 False player moved to 2 2
26 0 False player could not move
26 0 False player could not move
14 0 False player moved to 2 1
32 0 False player moved to 1 1
14 0 False player moved to 2 1
11 0 False player moved to 2 0
14 0 False player moved to 2 1
32 0 False player moved to 1 1
30 0 False player moved to 1 2
26 0 False player moved to 2 2
30 0 False player moved to 1 2
32 0 False player moved to 1 1
32 0 False player could not move
39 0 False player moved to 0 1
35 1 True hit the goal!
14 0 False player moved to 2 1
11 0 False player moved to 2 0
14 0 False player moved to 2 1
26 0 False player moved to 2 2
30 0 False player moved to 1 2
32 0 False player moved to 1 1
14 0 False player moved to 2 1
32 0 False player moved to 1 1
39 0 Fa

35 1 True hit the goal!
30 0 False player moved to 1 2
26 0 False player moved to 2 2
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
26 0 False player could not move
26 0 False player could not move
30 0 False player moved to 1 2
32 0 False player moved to 1 1
14 0 False player moved to 2 1
11 0 False player moved to 2 0
14 0 False player moved to 2 1
26 0 False player moved to 2 2
26 0 False player could not move
26 0 False player could not move
26 0 False player could not move
26 0 False player could not move
26 0 False player could not move
26 0 False player could not move
30 0 False player moved to 1 2
30 0 False player could not move
35 1 True hit the goal!
26 0 False player could not move
26 0 False player could not move
26 0 False player could not move
30 0 False player moved to 1 2
35 1 True hit the goal!
26 0 False player could not move
26 0 False player could not move
30 0 False player moved to 1 2
35 1 True hit t

30 0 False player moved to 1 2
35 1 True hit the goal!
26 0 False player could not move
30 0 False player moved to 1 2
30 0 False player could not move
35 1 True hit the goal!
30 0 False player moved to 1 2
32 0 False player moved to 1 1
32 0 False player could not move
30 0 False player moved to 1 2
30 0 False player could not move
26 0 False player moved to 2 2
14 0 False player moved to 2 1
26 0 False player moved to 2 2
30 0 False player moved to 1 2
30 0 False player could not move
35 1 True hit the goal!
26 0 False player could not move
30 0 False player moved to 1 2
30 0 False player could not move
26 0 False player moved to 2 2
26 0 False player could not move
14 0 False player moved to 2 1
14 0 False player could not move
26 0 False player moved to 2 2
26 0 False player could not move
30 0 False player moved to 1 2
30 0 False player could not move
35 1 True hit the goal!
30 0 False player moved to 1 2
26 0 False player moved to 2 2
30 0 False player moved to 1 2
35 1 True hit 

...

35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True 

30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False

35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
26 0 False player could not move
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 Fal

30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False

30 0 False player moved to 1 2
35 1 True hit the goal!
14 0 False player moved to 2 1
26 0 False player moved to 2 2
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 0 False player moved to 1 2
35 1 True hit the goal!
30 

In [28]:
# Calculate and print the average reward per thousand episodes
rewards_per_thosand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thosand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  1.0000000000000007
2000 :  1.0000000000000007
3000 :  1.0000000000000007
4000 :  1.0000000000000007
5000 :  1.0000000000000007
6000 :  1.0000000000000007
7000 :  1.0000000000000007
8000 :  1.0000000000000007
9000 :  1.0000000000000007
10000 :  1.0000000000000007


In [29]:
q_table

array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.95931076, 0.97029899, 0.95988879, 0.96013359],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.9800986 , 0.9801    , 0.97029896, 0.96059578],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        ,

In [30]:
# Watch it play!

In [ ]:
# Watch our agent play Frozen Lake by playing the best action 
# from each state according to the Q-table

for episode in range(3):
    # initialize new episode params
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):        
        # Show current state of environment on screen
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
    
        # Choose action with highest Q-value for current state       
        action = np.argmax(q_table[state,:])        

        # Take new action
        new_state, reward, done, info = env.step(action)
        
        if done:
            if reward == 1:
                # Agent reached the goal and won episode
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                # Agent stepped in a hole and lost episode   
                time.sleep(3)
                clear_output(wait=True)
            
        # Set new state
        state = new_state
        
env.close()

[1, 0, 5]
[6, 0, 0]
[0, 0, 0]
